# Suivi de trajectoire pour drone à voilure fixe

## Imports

In [11]:
import os
import pygame
from pygame.math import Vector2
from math import *
import numpy as np
import random as rd

## Constants

### Global constants

In [12]:
k_position = 1
k_velocity = 1
k_angle_penalty = 1
k_velocity_penalty = 1
k_theta = 1
k_omega = 1

width = 1280
height = 720

### Swarm constants

In [13]:
N_drones = 5
launch_position = np.zeros(2)
k_collision_angle = 0.0001
k_collision_acceleration = 0.001

In [14]:
def signed_angle(u, v):
    angle = atan2(v[1], v[0]) - atan2(u[1], u[0])
    angle = degrees(angle)
    if angle > 180:
        angle -= 360
    elif angle < -180:
        angle += 360
    return angle

def add_gaussian_noise(vec, mean=0, std_dev=1, size = 2):
    noise = np.random.normal(mean, std_dev, size)
    return vec + noise

In [15]:
'''
class Vehicle:
    def __init__(self, x, y, scaling=1, angle_0=0.0, length_0=4, max_steering_0=30, max_acceleration_0=5.0, k_steering_0=30, brake_deceleration_0=10, free_deceleration_0=2, max_velocity_0=20):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle_0
        self.length = length_0
        self.max_acceleration = max_acceleration_0
        self.max_steering = max_steering_0
        self.max_velocity = max_velocity_0/scaling
        self.brake_deceleration = brake_deceleration_0
        self.free_deceleration = free_deceleration_0
        self.k_steering = k_steering_0
        self.scaling = scaling

        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt):
        # User input
        pressed = pygame.key.get_pressed()

        if pressed[pygame.K_UP]:
            if self.velocity.x < 0:
                self.acceleration = self.brake_deceleration
            else:
                self.acceleration += (1/self.scaling) * dt
        elif pressed[pygame.K_DOWN]:
            if self.velocity.x > 0:
                self.acceleration = -self.brake_deceleration
            else:
                self.acceleration -= (1/self.scaling) * dt
        elif pressed[pygame.K_SPACE]:
            if abs(self.velocity.x) > dt * self.brake_deceleration:
                self.acceleration = -copysign(self.brake_deceleration, self.velocity.x)
            else:
                self.acceleration = -self.velocity.x / dt
        else:
            if abs(self.velocity.x) > dt * self.free_deceleration:
                self.acceleration = -copysign(self.free_deceleration, self.velocity.x)
            else:
                if dt != 0:
                    self.acceleration = -self.velocity.x / dt
        self.acceleration = max(-self.max_acceleration, min(self.acceleration, self.max_acceleration))

        #steering
        if pressed[pygame.K_RIGHT]:
            self.steering -= self.k_steering*dt
        elif pressed[pygame.K_LEFT]:
            self.steering += self.k_steering*dt
        else:
            self.steering = 0
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))

        #velocity update
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(-self.max_velocity, min(self.velocity.x, self.max_velocity))


        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt
    
    def draw(self, car_image, ppu, game):
        rotated_car = pygame.transform.rotate(car_image, self.angle)
        rect = rotated_car.get_rect()
        game.screen.blit(rotated_car, self.position * ppu - (rect.width / 2, rect.height / 2))
        return
'''

'\nclass Vehicle:\n    def __init__(self, x, y, scaling=1, angle_0=0.0, length_0=4, max_steering_0=30, max_acceleration_0=5.0, k_steering_0=30, brake_deceleration_0=10, free_deceleration_0=2, max_velocity_0=20):\n        self.position = Vector2(x, y)\n        self.velocity = Vector2(0.0, 0.0)\n        self.angle = angle_0\n        self.length = length_0\n        self.max_acceleration = max_acceleration_0\n        self.max_steering = max_steering_0\n        self.max_velocity = max_velocity_0/scaling\n        self.brake_deceleration = brake_deceleration_0\n        self.free_deceleration = free_deceleration_0\n        self.k_steering = k_steering_0\n        self.scaling = scaling\n\n        self.acceleration = 0.0\n        self.steering = 0.0\n\n    def update(self, dt):\n        # User input\n        pressed = pygame.key.get_pressed()\n\n        if pressed[pygame.K_UP]:\n            if self.velocity.x < 0:\n                self.acceleration = self.brake_deceleration\n            else:\n 

In [16]:
class Vehicle:
    def __init__(self, x, y, angle=0.0, length=4, max_steering=40, max_acceleration=0.0, scaling=1):
        self.position = Vector2(10, 10)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle
        self.length = length
        self.max_acceleration = max_acceleration
        self.max_steering = max_steering
        self.max_velocity = 2
        self.brake_deceleration = 1
        self.free_deceleration = 0.2

        self.steering_acc = 0.0
        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt):
        self.acceleration = rd.gauss(1,2)
        self.acceleration = max(-self.max_acceleration, min(self.acceleration, self.max_acceleration))

        self.steering_acc = rd.gauss(0,70)
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(-self.max_velocity, min(self.velocity.x, self.max_velocity))
        self.steering += self.steering_acc*dt
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))
        if abs(self.steering) == self.max_steering:
            self.steering = 0
        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt

# Single drone pursuit

In [17]:
class Drone:
    def __init__(self, x, y, scaling=1, angle_0=0, length_0=2, max_steering_0=30, max_acceleration_0=5.0, k_steering_0=30,
                 brake_deceleration_0=10, free_deceleration_0=2, max_velocity_0=20, min_velocity_0=0,
                 target_position_0=Vector2(0,0), swarm_index_0=0):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle_0
        self.length = length_0
        self.max_acceleration = max_acceleration_0
        self.max_steering = max_steering_0
        self.max_velocity = max_velocity_0/scaling
        self.min_velocity = min_velocity_0/scaling
        self.brake_deceleration = brake_deceleration_0
        self.free_deceleration = free_deceleration_0
        self.k_steering = k_steering_0
        self.swarm_index = swarm_index_0
        try:
            self.previous_theta = acos(Vector2.dot(target_position_0, Vector2(1.0, 0.0))/Vector2.magnitude(target_position_0)) - self.angle
        except:
            self.previous_theta = 0
        self.acceleration = 0.0
        self.steering = 0.0

    def angle_calculation(self, target_position):
        v = Vector2.normalize(target_position - self.position) # à bruiter parce qu'on connait pas la position exacte
        #v = add_gaussian_noise(v,mean=0,std_dev=0.5)
        v[1] = -v[1] # attention ca donne l'angle dans la base "écran" sinon
        u = Vector2(1.0, 0.0).rotate(self.angle) #obtenable avec un gyroscope
        
        theta = signed_angle(u, v)
        return theta

    def update(self, dt, target_position, target_velocity, drone_positions=[]):
        # Deltas
        del_position = Vector2.magnitude(target_position - self.position)
        del_velocity = Vector2.magnitude(target_velocity - self.velocity)

        theta = self.angle_calculation(target_position)

        #print(target_position.y, target_position.x)
        del_theta = (theta - self.previous_theta)

        self.previous_theta = theta

        collision_angle = 0
        collision_acceleration = 0

        # Avoiding collisions
        if (len(drone_positions) != 0):
            for i in range(len(drone_positions)):
                if (i != self.swarm_index):
                    # Angle
                    del_position = Vector2.magnitude(drone_positions[i] - self.position)
                    angle = self.angle_calculation(drone_positions[i])

                    if (self.position != drone_positions[i]):
                        angle = self.angle_calculation(drone_positions[i])
                        if ((angle >= -90) & (angle <= 90)):
                            #if a drone is on the right, turn left
                            collision_angle += (1.0/del_position)*cos(angle*pi/180.0)
                        else:
                            #if a drone is on the left, turn right
                            collision_angle -= (1.0/del_position)*cos(angle*pi/180.0)

                    # Acceleration
                    if ((angle >= 0) & (angle <= 180)):
                        #if a drone is in front, decelerate
                        collision_acceleration -= (1.0/del_position)*sin(angle*pi/180.0)
                    else:
                        #if a drone is behind, accelerate
                        collision_acceleration += (1.0/del_position)*sin(angle*pi/180.0)

        #Gaz pedal controller
        self.acceleration = k_position*del_position + k_velocity*del_velocity + k_collision_acceleration*collision_acceleration
        #self.acceleration = 0

        #steering controller
        self.steering = k_theta*theta + k_omega*del_theta + k_collision_angle*collision_angle - k_velocity_penalty*Vector2.magnitude(self.velocity)/self.max_velocity
        self.steering = k_theta*theta
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))

        #velocity update
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(self.min_velocity, min(self.velocity.x, self.max_velocity))


        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt

In [18]:
def draw(object, image, ppu, game):
    rotated_drone = pygame.transform.rotate(image, object.angle)
    rect = rotated_drone.get_rect()
    position = object.position * ppu - (rect.width / 2, rect.height / 2)
    position = [position[0]%width,position[1]%height]
    game.screen.blit(rotated_drone, position)
    return

In [19]:
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Drone pursuit")
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        car_image_path = "car.png"
        drone_image_path = "drone.png"
        car_image = pygame.image.load(car_image_path)
        drone_image = pygame.image.load(drone_image_path)
        taille_car = car_image.get_size()
        scaling_car = 10
        taille_car = [taille_car[0]/scaling_car, taille_car[1]/scaling_car]
        car_image = pygame.transform.scale(car_image, taille_car)

        taille_drone = drone_image.get_size()
        scaling_drone = 10
        taille_drone = [taille_drone[0]/scaling_drone, taille_drone[1]/scaling_drone]

        car_image = pygame.transform.scale(car_image, taille_car)
        drone_image = pygame.transform.scale(drone_image, taille_drone)

        car = Vehicle(x=10, y=10, scaling=scaling_car)
        drone = Drone(x=5, y=20, scaling=scaling_drone)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # Logic
            car.update(dt)

            drone.update(dt, target_position=car.position, target_velocity=car.velocity)
            
            # Drawing
            self.screen.fill((0, 0, 0))
            draw(car, car_image, ppu, self)
            draw(drone, drone_image, ppu, self)
            
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()

## Drone swarm pursuit

### Swarm class

In [21]:
class Swarm():
    def __init__(self, N_drones_0, launch_position_0, scaling_drone=1):
        self.swarm_list = []
        self.drone_positions = []
        self.N_drones = N_drones_0
        for i in range(self.N_drones):
            position = launch_position_0 + np.random.normal(loc=0, scale=100, size=2)
            x = position[0]
            y = position[1]
            self.swarm_list.append(Drone(x=x, y=y, scaling=scaling_drone, swarm_index_0=i))
            self.drone_positions.append(self.swarm_list[i].position)

    def update(self, dt, target_position, target_velocity):
        for i in range(self.N_drones):
            self.swarm_list[i].update(dt, target_position, target_velocity, self.drone_positions)
            self.drone_positions[i] = self.swarm_list[i].position

    def draw(self, drone_image, ppu, game):
        for i in range(self.N_drones):
            draw(self.swarm_list[i], drone_image, ppu, game)

### Swarm game

In [25]:
class Swarm_game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Drone pursuit")
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        car_image_path = "car.png"
        drone_image_path = "drone.png"
        car_image = pygame.image.load(car_image_path)
        drone_image = pygame.image.load(drone_image_path)
        taille_car = car_image.get_size()
        scaling_car = 10
        taille_car = [taille_car[0]/scaling_car, taille_car[1]/scaling_car]
        car_image = pygame.transform.scale(car_image, taille_car)

        taille_drone = drone_image.get_size()
        scaling_drone = 10
        taille_drone = [taille_drone[0]/scaling_drone, taille_drone[1]/scaling_drone]

        car_image = pygame.transform.scale(car_image, taille_car)
        drone_image = pygame.transform.scale(drone_image, taille_drone)

        car = Vehicle(x=10, y=10, scaling=scaling_car)
        swarm = Swarm(N_drones_0=N_drones, scaling_drone=scaling_drone, launch_position_0=launch_position)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # Logic
            car.update(dt)

            swarm.update(dt, target_position=car.position, target_velocity=car.velocity)
            
            # Drawing
            self.screen.fill((0, 0, 0))
            draw(car, car_image, ppu, self)
            swarm.draw(drone_image, ppu, self)
            
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()

## Run

### Swarm simulation

In [26]:
game = Swarm_game()
game.run()

KeyboardInterrupt: 

### Single drone simulation

In [ ]:
game = Game()
game.run()

63.43494882292201
63.43494882292201
63.4034787326175
63.3455119073098
63.25141076636236
63.12424328006774
62.96344304056246
62.76843073238718
62.554048630801574
62.308327940746274
62.011306177760545
61.679296439856685
61.34703109971784
61.034053980728565
60.72084354611218
60.38781964809152
60.07411743678172
59.74056393306475
59.40673157973681
59.07261625229581
58.7382137855082
58.403519972668164
58.088222126953006
57.75295065111675
57.41737520877323
57.101235226257735
56.76505724946593
56.42856230058171
56.09174586639055
55.77442074583332
55.43696723879266
55.11903382120116
54.7809256925903
54.44247334133877
54.123586318352814
53.784452045329324
53.44495926008476
53.10510299740396
52.76487823625725
52.444299387985524
52.12338513485295
51.78206968833517
51.46044990572733
51.13848131344809
50.79603157150326
50.47332934222056
50.15026444298785
49.826832245295634
49.48280799370962
49.138358386545235
48.81374792647554
48.4887505175003
48.14304253558827
47.81723147600546
47.491018269818845
4

KeyboardInterrupt: 